## Selección de Variables Numéricas

In [1]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression,  f_classif, mutual_info_classif, chi2

In [ ]:
df_2015 = 
df_2016 = 

EmployeeID, DateSurvey, EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance

EmployeeID, InfoDate, Age, BusinessTravel, Department, DistanceFromHome, Education, 
    JobRole, MonthlyIncome, NumCompaniesWorked, PercentSalaryHike, TrainingTimesLastYear, 
    YearsAtCompany, YearsSinceLastPromotion

EmployeeID, SurveyDate, JobInvolvement, PerformanceRating

EmployeeID, retirementDate, retirementType


## Métodos de Filtrado


Umbral de Varianza

In [ ]:
x_numericos = df.select_dtypes(include = ["number"]) # solo variables númericas
x_numericos.head()

In [ ]:
x_numericos.describe()

In [ ]:
# Función de filtro de caracteristicas
def variance_threshold(X,th): # recibe el dataframe y el umbral
    var_thres = VarianceThreshold(threshold=th) # crea la función con base en el umbral
    var_thres.fit(X) # alimenta los datos con la función creada
    new_cols = var_thres.get_support() # devuelve las columnas
    return new_cols

In [ ]:
# Normalizar los datos
df1 = x_numericos.copy(deep = True)  # crear una copia del DataFrame
scaler = MinMaxScaler() # asignar el tipo de normalización
sv = scaler.fit_transform(df1.iloc[:,:]) # normalizar los datos
df1.iloc[:,:] = sv # asignar los nuevos datos
df1.head()

In [ ]:
df1.describe() # miremos sus metricas

In [ ]:
# Obtener columnas seleccionadas
import math as mt

# Parametros para definir que era constante:
desviacion = 0.21 # eliminar todo cuya desviación estandar sea menor a 0.21, teniendo en cuenta tabla anterior
varianza = desviacion**2

# Resultados
print('Desviación estandar:', mt.sqrt(varianza))
print('Varianza:', varianza)
x_new = variance_threshold(df1, varianza)
df_new_int = df1.iloc[:,x_new] # nuevo dataframe
df_new_int.head()

Selección Univariante

In [ ]:
x_num = df.select_dtypes(include = ["number"]) # filtrar solo variables númericas
y = df['Credit Risk']
x_num.head()

In [ ]:
# Normalización de datos
x_num_norm = x_num.copy(deep = True)  # copia
scaler = MinMaxScaler() # Se usará minmaxScaler
sv = scaler.fit_transform(x_num_norm.iloc[:,:]) # normalizar los datos
x_num_norm.iloc[:,:] = sv # asignar los nuevos datos
x_num_norm.head()

In [ ]:
# Función de filtro de caracteristicas - stadis. scores
def select_kbest(X,y,score_f,k): #se establece una funcion que permite sacar varias funciones de evaluacion
    sel_kb = SelectKBest(score_func=score_f, k=k)
    sel_kb.fit(X,y)
    scores = sel_kb.scores_
    pvalues = sel_kb.pvalues_
    new_cols = sel_kb.get_support()
    print("Scores:\n", scores, "\nP-values:\n", pvalues)

In [ ]:
#F Classif 
x_new, scores, pvalues = select_kbest(x_num_norm, y,  f_classif, 3) # Obtener columnas seleciconadas - (3 caracteristicas)
df_new = x_num.iloc[:,x_new] # Nuevo conjunto de datos
df_new.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo


In [ ]:
def plot(metric, features, name):
    features = features
    fig, ax = plt.subplots(figsize=(10, 6))  # Ajuste el tamaño de la figura según sea necesario
    width = 0.4  # Ancho de las barras
    ax.bar(np.arange(len(metric)), metric, width=width)
    ax.set_xlabel('Features')
    ax.set_ylabel(name)
    ax.set_xticks(np.arange(len(metric)))
    ax.set_xticklabels(features)
    plt.xticks(rotation=45, ha='right')  # Rotar etiquetas para mejor visualización
    plt.tight_layout()  # Ajustar el diseño para evitar superposiciones
    return plt.show()

In [ ]:
plot(scores, x_num.columns, 'F-values')
plot(pvalues, x_num.columns, 'P-values')

## Métodos Wrapper 

RFE (Recursive Feature Elimination)

In [ ]:
# Función recursiva de selección de características
def recursive_feature_selection(X,y,model,k): #model=modelo que me va a servir de estimador en este caso de regresión logística
  rfe = RFE(model, n_features_to_select=k, step=1)# step=1 cada cuanto el toma la sucesión de tomar una caracteristica
  fit = rfe.fit(X, y)
  X_new = fit.support_
  print("Num Features: %s" % (fit.n_features_))
  print("Selected Features: %s" % (fit.support_))
  print("Feature Ranking: %s" % (fit.ranking_))

  return X_new

In [ ]:
#  Estimador en este caso para regresión logística (problema de clasificación)
model = LogisticRegressionCV()

# Obtener columnas seleciconadas - (4 caracteristicas)
X_new = recursive_feature_selection(x_num_norm, y, model, 4)

# Nuevo conjunto de datos
df_new = x_num.iloc[:,X_new]
df_new.head()

## Métodos Integrados 

Regresión Ridge

In [ ]:
# Selector con un modelo de regresión logística
sel_ = SelectFromModel(LogisticRegression(penalty='l2', C=1.0))  # Usamos 'l2' para regularización Ridge

sel_.fit(x_num_norm, y)

# Obtener variables seleccionadas
X_new = sel_.get_support()

df_new = x_num.iloc[:, X_new]
print(df_new.head())